In [53]:
#%%writefile fantasybasketball.py
"""
----------------------------------------------------------------------
Welcome to Jay's amazing Fantasy Basketball lineup optimizer!
Please put the main team you would like analyzed in a file called
primaryteam.txt.  It should be formatted as a comma-delimited file
with name, FPPMty, min per game, typical # of games per season,
primary, and secondary position, if applicable.
For example: "Stephen Curry, 1.24, 78, G"
If you would like to compare several teams, provide addl teams as
text files in the application directory in the same format.
Also, if you would like to change the number of simulated games per
lineup, please provide an integer after the argument "-sims.
For example: "python fantasybasketball.py -sims 1000"
----------------------------------------------------------------------
"""
import argparse
import sys
import os
import random
import csv


class clsPlayer:
    def __init__(self, Name, FPPM, MPG, GamesTypical, PrimaryPos,
                 SecondaryPos='  ', PrimaryMins=0, SecondaryMins=0,
                 PrimaryPts=0, SecondaryPts=0, simMPG=0):

        # Go ahead and have exceptions bubble up if formatting is wrong.
        self._Name = str(Name)  # Player's Name
        self._FPPM = float(FPPM)  # Expected fantasy points per minute
        self._MPG = float(MPG)  # Expected minutes per game

        if int(float(GamesTypical)) > 0:
            # Expected number of healthy games per year (max 82)
            self._GamesTypical = int(float(GamesTypical))
        else:
            self._GamesTypical = 75

        # Primary position
        self._PrimaryPos = str(PrimaryPos).replace("\n", "").strip()
        # Secondary position if applicable
        self._SecondaryPos = str(SecondaryPos).replace("\n", "").strip()

        if self._SecondaryPos == "":
            self._SecondaryPos = " "  # for formatting reasons

        # Simulation mins played at primary position
        self._PrimaryMins = PrimaryMins
        # Simulation mins played at secondary pos
        self._SecondaryMins = SecondaryMins
        # Simulation pts scored at primary position
        self._PrimaryPts = PrimaryPts
        # Simulation pts scored at secondary pos
        self._SecondaryPts = SecondaryPts
        # Simulation minutes played in a particular game
        self._simMPG = simMPG
        self._Order = 0  # Used for shuffling order of players

    def setFPPM(self, FPPM=0):
        if FPPM > 0:
            self._FPPM = FPPM
        else:
            self._FPPM = 0

    def getFPPM(self):
        return self._FPPM

    def setMPG(self, MPG=0):
        if MPG > 0:
            self._MPG = MPG
        else:
            self._MPG = 0

    def getMPG(self):
        return self._MPG

    def setsimMPG(self, simMPG=0):
        if simMPG > 0:
            self._simMPG = simMPG
        else:
            self._simMPG = 0

    def getsimMPG(self):
        return self._simMPG

    def setGamesTypical(self, GamesTypical=0):
        if GamesTypical > 0:
            self._GamesTypical = GamesTypical
        else:
            self._GamesTypical = 0

    def getGamesTypical(self):
        return self._GamesTypical

    def setOrder(self, Order=0):
        if Order > 0:
            self._Order = Order
        else:
            self._Order = 0

    def getOrder(self):
        return self._Order

    def setPrimaryPos(self, PrimaryPos=0):
        self._PrimaryPos = PrimaryPos

    def getPrimaryPos(self):
        return self._PrimaryPos

    def setPrimaryMins(self, PrimaryMins=0):
        if PrimaryMins > 0:
            self._PrimaryMins = PrimaryMins
        else:
            self._PrimaryMins = 0

    def getPrimaryMins(self):
        return self._PrimaryMins

    def setPrimaryPts(self, PrimaryPts=0):
        if PrimaryPts > 0:
            self._PrimaryPts = PrimaryPts
        else:
            self._PrimaryPts = 0

    def getPrimaryPts(self):
        return self._PrimaryPts

    def setSecondaryPos(self, SecondaryPos=0):
        self._SecondaryPos = SecondaryPos

    def getSecondaryPos(self):
        return self._SecondaryPos

    def setSecondaryMins(self, SecondaryMins=0):
        if SecondaryMins > 0:
            self._SecondaryMins = SecondaryMins
        else:
            self._SecondaryMins = 0

    def getSecondaryMins(self):
        return self._SecondaryMins

    def setSecondaryPts(self, SecondaryPts=0):
        if SecondaryPts > 0:
            self._SecondaryPts = SecondaryPts
        else:
            self._SecondaryPts = 0

    def getSecondaryPts(self):
        return self._SecondaryPts

    def __repr__(self):
        # This looks ugly, but the result is a readable table with
        # name, positions, FPPM, MPG, and pts and mins by position
        positions = "(" + self._PrimaryPos + self._SecondaryPos + ")"
        FPPM = " [" + str("{0:.2f}".format(self._FPPM)) + "]  "
        MPG = str("{0:.2f}".format(self._MPG)).rjust(5) + " mpg  "
        Primary = "  PP: " + \
            str("{0:.1f}".format(self._PrimaryPts)).rjust(4) \
            + "  PM: " + str("{0:.1f}".format(self._PrimaryMins)).rjust(4)
        Secondary = "  SP: " + \
            str("{0:.1f}".format(self._SecondaryPts)).rjust(4) + \
            "  SM: " + str("{0:.1f}".format(self._SecondaryMins)).rjust(4)

        niceFormat = self._Name.ljust(25) + positions + FPPM + \
            MPG + Primary + Secondary

        return niceFormat


class clsTeam:
    def __init__(self, Primary=True, ComparisonTeamFile=''):

        nope = False  # If something goes wrong, this will be set to True

        if Primary:
            self._Name = "Primary"
        else:
            self._Name = ComparisonTeamFile.replace(".txt", "")

        self._Players = []  # Store the original lineup
        self._PlayersBenchmark2 = []  # Sorted by FPPM descending
        self._BestLineup = []  # Store lineup that resulted in high score
        self._TempLineup = []  # For use during simulation
        self._MaxAvg = 0  # Highest average score with optimized lineup
        self._OptUnusedC = 0  # Average number of Center minutes unused (bad)
        self._OptUnusedF = 0  # Ditto for Forward
        self._OptUnusedG = 0  # Ditto for Guard
        self._Benchmark1 = 0  # The score for the original lineup order
        self._B1UnusedC = 0  # Average number of Center minutes unused (bad)
        self._B1UnusedF = 0  # Ditto for Forward
        self._B1UnusedG = 0  # Ditto for Guard
        self._Benchmark2 = 0  # The score for a simple FPPM sort order
        self._B2UnusedC = 0  # Average number of Center minutes unused (bad)
        self._B2UnusedF = 0  # Ditto for Forward
        self._B2UnusedG = 0  # Ditto for Guard
        self._Primary = Primary  # True if this is the primary team

        if Primary:
            # Find PrimaryTeam.txt and use it to populate the team
            try:
                f = open("primaryteam.txt", mode='rt')
            except:
                print("Please put the primaryteam.txt file in the same \
                    directory as this program.")

        elif not Primary:
            if ComparisonTeamFile == '':
                print("Please provide a filename for the comparison team")
            else:
                try:
                    f = open(ComparisonTeamFile, mode='rt')
                except:
                    print(ComparisonTeamFile, "cannot be loaded.")

        csvin = csv.reader(f)

        for row in csvin:
            try:
                if len(row) > 5:  # Secondary position provided
                    # create a player object to be added to the Players list
                    print("Adding ",
                          row[0], row[1], row[2], row[3], row[4], row[5])
                    objPlayer = clsPlayer(
                        row[0], row[1], row[2], row[3], row[4], row[5])
                elif len(row) > 4:  # Primary position provided, only
                    # create a player object to be added to the Players list
                    print("Adding ",
                          row[0], row[1], row[2], row[3], row[4])
                    objPlayer = clsPlayer(
                        row[0], row[1], row[2], row[3], row[4])
                else:
                    raise

            except:
                print("\n", f.name, " appears to be incorrectly formatted.\n",
                      sys.exc_info()[0])
                nope = True
                break

            self._Players.append(objPlayer)

        if not nope:  # it imported okay
            if Primary:
                print("Primary team added.")
            else:
                print("Comparison team added.")

            # Order team by FPPM desc and store it in PlayersBenchmark2 list
            ListForSorting = []  # Store FPPM for each player
            for objPlayer in self._Players:
                ListForSorting.append(objPlayer.getFPPM())

            ListForSorting = list(set(ListForSorting))  # Ditch dupes
            ListForSorting.sort(reverse=True)

            # Create a team lineup where players are in FPPM order
            for FPPM in ListForSorting:
                for objPlayer in self._Players:
                    if objPlayer.getFPPM() == FPPM:
                        self._PlayersBenchmark2.append(objPlayer)

    def getPrimary(self):
        return self._Primary

    def getBestLineup(self):
        return self._BestLineup

    def getName(self):
        return self._Name

    def getBenchmark1(self):
        return self._Benchmark1

    def getBenchmark2(self):
        return self._Benchmark2

    def getMaxAvg(self):
        return self._MaxAvg

    def getOptUnusedC(self):
        return self._OptUnusedC

    def getOptUnusedF(self):
        return self._OptUnusedF

    def getOptUnusedG(self):
        return self._OptUnusedG

    def VaryMinutes(self, Team):
        """
        This varies minutes played in simulated games.  It also
        will cause players to be randomly injured depending on how
        many games out of 82 they typically play in a season.
        """

        for objPlayer in Team:
            # Normal mpg is a mean of 20.7 with a stan dev of 7.21
            # so let's vary mpg by plus/minus 7
            if objPlayer.getMPG() >= 7:
                RandomChange = random.randint(-7, 7)
            else:
                # Can't have negative minutes, vary between 0 and 2*mpg
                RandomChange = random.randint(0-int(objPlayer.getMPG()),
                                              int(objPlayer.getMPG()))

            objPlayer.setsimMPG(objPlayer.getMPG() + RandomChange)

            # Simulate injuries an appropriate percentage of the time
            rndNum = random.randint(1, 82)
            if rndNum > objPlayer.getGamesTypical():
                objPlayer.setsimMPG(0)

    def Shuffle(self, Team):
        """
        Randomize player order and position during simulations
        so that optimal lineups can be discovered.
        """

        sortedTeam = []
        initialorder = 0
        # Assume starting with FPPM ordered team, since the only
        # re-ordering that makes sense is how far down to put the
        # multi-position players.
        for objPlayer in Team:
            objPlayer.setOrder(initialorder)
            initialorder += 1

        for objPlayer in Team:
            if objPlayer.getSecondaryPos() != ' ':

                # It's okay that this will never put a multi-position
                # player first in the order, since being in the second
                # position still ensures that all minutes will be used.
                objPlayer.setOrder(random.random() * 13)

                # Randomly swap primary and secondary position
                if random.randint(0, 1) == 1:
                    tempPos = objPlayer.getPrimaryPos()
                    objPlayer.setPrimaryPos(objPlayer.getSecondaryPos())
                    objPlayer.setSecondaryPos(tempPos)

        # Sort by FPPM desc if positions are identical...
        for objPlayer1 in Team:
            for objPlayer2 in Team:
                if objPlayer1.getPrimaryPos() == \
                    objPlayer2.getPrimaryPos() and \
                    objPlayer1.getSecondaryPos() == \
                    objPlayer2.getSecondaryPos() and \
                        objPlayer1.getFPPM() > objPlayer2.getFPPM():
                    if objPlayer1.getOrder() > objPlayer2.getOrder():
                        objPlayer1.setOrder(objPlayer2.getOrder() - 0.000001)

        sortedTeam = sorted(self._Players, key=lambda
                            player: player.getOrder())

        return sortedTeam

    def UseUpMinutes(self, objPlayer, GuardMins, ForwardMins, CenterMins):
        """
        If a player has primary position minutes available, spend them
        up to the provided maximum minutes available for that position.
        If that maximum has already been met, see if the player has minutes
        at a secondary position that can be used.
        """

        # These stats will be recalculated
        objPlayer.setPrimaryMins(0)
        objPlayer.setSecondaryMins(0)

        objPlayer.setPrimaryPts(0)
        objPlayer.setSecondaryPts(0)

        MinsLeft = objPlayer.getsimMPG()

        # Use primary position minutes if possible
        # Guards
        if objPlayer.getPrimaryPos() == "G":
            if GuardMins <= MinsLeft:
                objPlayer.setPrimaryMins(
                    objPlayer.getPrimaryMins() + GuardMins)
                GuardMins = 0
            elif GuardMins > MinsLeft:
                objPlayer.setPrimaryMins(
                    objPlayer.getPrimaryMins() + MinsLeft)
                GuardMins = GuardMins - MinsLeft
        # Forwards
        elif objPlayer.getPrimaryPos() == "F":
            if ForwardMins <= MinsLeft:
                objPlayer.setPrimaryMins(
                    objPlayer.getPrimaryMins() + ForwardMins)
                ForwardMins = 0
            elif ForwardMins > MinsLeft:
                objPlayer.setPrimaryMins(
                    objPlayer.getPrimaryMins() + MinsLeft)
                ForwardMins = ForwardMins - MinsLeft
        # Centers
        elif objPlayer.getPrimaryPos() == "C":
            if CenterMins <= MinsLeft:
                objPlayer.setPrimaryMins(
                    objPlayer.getPrimaryMins() + CenterMins)
                CenterMins = 0
            elif CenterMins > MinsLeft:
                objPlayer.setPrimaryMins(
                    objPlayer.getPrimaryMins() + MinsLeft)
                CenterMins = CenterMins - MinsLeft

        # Primary minutes may have changed above
        MinsLeft = objPlayer.getsimMPG() - objPlayer.getPrimaryMins()

        if MinsLeft > 0:
            # More minutes are available at secondary position
            if objPlayer.getSecondaryPos() == "G":
                if GuardMins <= MinsLeft:
                    objPlayer.setSecondaryMins(
                        objPlayer.getSecondaryMins() + GuardMins)
                    GuardMins = 0
                elif GuardMins > MinsLeft:
                    GuardMins = GuardMins - MinsLeft
                    objPlayer.setSecondaryMins(
                        objPlayer.getSecondaryMins() + MinsLeft)
            if objPlayer.getSecondaryPos() == "F":
                if ForwardMins <= MinsLeft:
                    objPlayer.setSecondaryMins(
                        objPlayer.getSecondaryMins() + ForwardMins)
                    ForwardMins = 0
                elif ForwardMins > MinsLeft:
                    ForwardMins = ForwardMins - MinsLeft
                    objPlayer.setSecondaryMins(
                        objPlayer.getSecondaryMins() + MinsLeft)
            if objPlayer.getSecondaryPos() == "C":
                if CenterMins <= MinsLeft:
                    objPlayer.setSecondaryMins(
                        objPlayer.getSecondaryMins() + CenterMins)
                    CenterMins = 0
                elif CenterMins > MinsLeft:
                    CenterMins = CenterMins - MinsLeft
                    objPlayer.setSecondaryMins(
                        objPlayer.getSecondaryMins() + MinsLeft)

        return GuardMins, ForwardMins, CenterMins

    def ScoreTeam(self, WhichTeam, gintIterations):
        """
        This function organizes the team in different ways and tracks
        the expected average score for those lineups.
        WhichTeam == "B1" is the original order and "B2" considers
        the team in FPPM order.  Otherwise, try a pretty random order.
        """

        # These will track the total stats to be averaged
        TotalUnusedC = 0
        TotalUnusedF = 0
        TotalUnusedG = 0
        TotalScore = 0

        if WhichTeam == "B1":
            Team = self._Players
            if self._Primary:
                print("\nPrimary Team, original order...")
            else:
                print("\nTeam " + self._Name + ", original order...")

        elif WhichTeam == "B2":
            Team = self._PlayersBenchmark2
            if self._Primary:
                print("\nPrimary Team, FPPM order...")
            else:
                print("\nTeam " + self._Name + ", FPPM order...")

        else:  # Try a random lineup
            # start with a copy of FPPM ordered team
            self._TempLineup = list(self._PlayersBenchmark2)
            self._TempLineup = self.Shuffle(self._TempLineup)
            Team = self._TempLineup

        for n in range(gintIterations):
            # Each iteration = one game
            curTeamScore = 0  # Each game's score

            GuardMins = 96  # Minutes to be allocated by position
            ForwardMins = 96
            CenterMins = 48

            self.VaryMinutes(Team)  # Sets simMPG

            for objPlayer in Team:
                # Determine how many mins each player plays at each position
                # and decrement the remaining mins allowable by position
                GuardMins, ForwardMins, CenterMins = \
                    self.UseUpMinutes(objPlayer, GuardMins, ForwardMins,
                                      CenterMins)

                # Calculate points scored by assuming FPPM is constant
                objPlayer.setPrimaryPts(
                    objPlayer.getFPPM() * objPlayer.getPrimaryMins())
                objPlayer.setSecondaryPts(
                    objPlayer.getFPPM() * objPlayer.getSecondaryMins())

                # Add the player's point totals to the team score
                curTeamScore = curTeamScore + objPlayer.getPrimaryPts() + \
                    objPlayer.getSecondaryPts()

            # Track how many minutes were unused (bad)
            TotalUnusedC = TotalUnusedC + CenterMins
            TotalUnusedF = TotalUnusedF + ForwardMins
            TotalUnusedG = TotalUnusedG + GuardMins
            TotalScore += curTeamScore

        # Track the average number of unused minutes
        UnusedC = round(TotalUnusedC / gintIterations, 1)
        UnusedF = round(TotalUnusedF / gintIterations, 1)
        UnusedG = round(TotalUnusedG / gintIterations, 1)

        if WhichTeam != "Opt":
            # Store average scores and unused minutes for later
            if WhichTeam == "B1":
                self._Benchmark1 = round(TotalScore / gintIterations, 1)
                self._B1UnusedC = UnusedC
                self._B1UnusedF = UnusedF
                self._B1UnusedG = UnusedG

            elif WhichTeam == "B2":
                self._Benchmark2 = round(TotalScore / gintIterations, 1)
                self._B2UnusedC = UnusedC
                self._B2UnusedF = UnusedF
                self._B2UnusedG = UnusedG
        else:
            if TotalScore / gintIterations > self._MaxAvg:
                self._MaxAvg = round(TotalScore / gintIterations, 1)
                print("\nNew high score! " + str(self._MaxAvg) + " points!")
                self._BestLineup = list(Team)
                self._OptUnusedC = UnusedC
                self._OptUnusedF = UnusedF
                self._OptUnusedG = UnusedG

    def DisplayTeam(self, WhichTeam, Iterations):
        """
        This function can display any of the three lineups, showing the
        order of players, the primary/secondary positions, and the use
        of minutes and points scored in the last simulated game for
        debugging purposes.
        WhichTeam must be "B1", "B2", or "Opt" for Benchmark 1, 2,
        or Optimized lineups, respectively.
        """

        UnusedC = 0
        UnusedF = 0
        UnusedG = 0
        AvgScore = 0

        if WhichTeam == 'B1' or WhichTeam == 'B2' or WhichTeam == 'Opt':
            if WhichTeam == 'B1':
                objTeam = self._Players
                AvgScore = self._Benchmark1
                UnusedC = self._B1UnusedC
                UnusedF = self._B1UnusedF
                UnusedG = self._B1UnusedG

            elif WhichTeam == 'B2':
                objTeam = self._PlayersBenchmark2
                AvgScore = self._Benchmark2
                UnusedC = self._B2UnusedC
                UnusedF = self._B2UnusedF
                UnusedG = self._B2UnusedG

            elif WhichTeam == 'Opt':
                objTeam = self._BestLineup
                AvgScore = self._MaxAvg
                UnusedC = self._OptUnusedC
                UnusedF = self._OptUnusedF
                UnusedG = self._OptUnusedG

            for objPlayer in objTeam:
                print(objPlayer)

            print("")
            print("Simulations: " + str(Iterations))
            print("Avg Team Score: " + str(AvgScore))
            print("Avg Unused C Mins: " + str(UnusedC))
            print("Avg Unused F Mins: " + str(UnusedF))
            print("Avg Unused G Mins: " + str(UnusedG))
            print("")


class clsComparison:
    def __init__(self):
        self._Teams = []

    def ScoreLineups(self, Iterations=200):
        if Iterations is not None and Iterations > 0:
            gintIterations = Iterations  # number of simulated games per lineup
        else:
            gintIterations = 200

        gintLineupShuffles = 50  # likely to hit all possibilities?

        for objTeam in self._Teams:
            # Benchmark 1 = original order
            objTeam.ScoreTeam("B1", gintIterations)
            objTeam.DisplayTeam("B1", gintIterations)

            # Benchmark 2 = FPPM sorted order
            objTeam.ScoreTeam("B2", gintIterations)
            objTeam.DisplayTeam("B2", gintIterations)

            for i in range(gintLineupShuffles):
                print(".", end="")
                objTeam.ScoreTeam("Opt", gintIterations)

            if objTeam.getPrimary():
                print("\nPrimary Team Simulation")
            else:
                print("\nTeam " + objTeam.getName() + " Simulation")

            objTeam.DisplayTeam("Opt", gintIterations)

        # Print summary statistics
        print("\nSummary" + "-" * 87)
        print("Team".ljust(20) + "Orig Order".ljust(15) +
              "FPPM Order".ljust(15) + "Opt Order".ljust(15) +
              "Unused C".ljust(10) + "Unused F".ljust(10) +
              "Unused G".ljust(10))

        for objTeam in self._Teams:
            print(objTeam.getName().ljust(20) +
                  str(objTeam.getBenchmark1()).ljust(15) +
                  str(objTeam.getBenchmark2()).ljust(15) +
                  str(objTeam.getMaxAvg()).ljust(15) +
                  str(objTeam.getOptUnusedC()).ljust(10) +
                  str(objTeam.getOptUnusedF()).ljust(10) +
                  str(objTeam.getOptUnusedG()).ljust(10))

    def ImportTeams(self, Iterations=0):
        files = [f for f in os.listdir('.')]
        for f in files:
            if f.endswith(".txt"):
                if f.lower() == "primaryteam.txt":
                    objPrimaryTeam = clsTeam()  # Auto imports primary.txt
                    self._Teams.append(objPrimaryTeam)
                else:
                    objTeam = clsTeam(Primary=False, ComparisonTeamFile=f)
                    if len(objTeam._Players) > 0:  # Bad file format = no team
                        self._Teams.append(objTeam)

        if len(self._Teams) > 0:
            self.ScoreLineups(Iterations)
        else:
            print("No teams found.  Please at least put a primaryteam.txt \
file in the application directory.")

# Code starts here
if __name__ == "__main__":
    if sys.argv[0].find('ipykernel') != -1:
        # only run this if program is run from the command-line
        parser = argparse.ArgumentParser(
            description='Grab number of simulations per lineup')
        parser.add_argument(
            '-sims', type=int, help='number of simulated games per lineup')

        print(__doc__)  # Friendly header

        args = parser.parse_args()

        # Create comparison object, import teams, and run analysis
        compare = clsComparison()
        compare.ImportTeams(args.sims)

    else:
        # For running in an iPython notebook
        compare = clsComparison()
        compare.ImportTeams()

Adding  Hassan Whiteside  0.99  28  75  C
Adding  Rajon Rondo  0.80  35  75  G
Adding  Deron Williams  0.67  32  75  G
Adding  Patty Mills  0.67  20  75  G
Adding  Leandro Barbosa  0.63  15  75  G
Adding  Shabazz Napier  0.56  12  75  G
Adding  Anthony Davis  1.07  36  75  C  F
Adding  Luis Scola  0.69  21  75  C  F
Adding  JaMychal Green  0.69  17  75  F
Adding  Jonas Jerebko  0.60  14  75  F
Adding  Nemanja Bjelica  0.52  17  75  F
Adding  Andre Roberson  0.46  22  75  F  G
Comparison team added.
Adding  Michael Beasley  1.37  10.2  70  F
Adding  Stephen Curry  1.25  30.7  78  G
Adding  DeMarcus Cousins  1.19  27.1  70  C
Adding  Russell Westbrook  1.19  24.3  70  G
Adding  Kevin Durant  1.11  35.6  70  F
Adding  James Harden  1.07  37.2  70  G
Adding  LeBron James  1.06  35.4  70  F
Adding  Anthony Davis  1.06  35.4  70  F  C
Adding  Blake Griffin  1.04  34.5  70  F
Adding  Pau Gasol  1.03  31.6  70  F  C
Adding  Andre Drummond  1.02  32.8  70  C
Adding  Chris Paul  1.02  32.8  70  

In [116]:
def solvepuzzle():

    import itertools
    
    iter = itertools.permutations([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])
    num = ""
    
    for i in iter:
        num = str(i[0]*10**9 + i[1]*10**8 + i[2]*10**7 + i[3]*10**6 + \
                  i[4]*10**5 + i[5]*10**4 + i[6]*10**3 + i[7]*10**2 + i[8]*10 + i[9])
    
        if len(num) < 10:
            continue
        if num[0] !="0" and int(num[0:2]) % 2 == 0 and int(num[0:3]) % 3 == 0 and int(num[0:4]) % 4 == 0 \
        and num[4] == "5" and int(num[0:6]) % 6 == 0 and \
        int(num[0:7]) % 7 == 0 and int(num[0:8]) % 8 == 0 and int(num[0:9]) % 9 == 0 and num[9] == "0":
            return num


In [112]:
print(solvepuzzle())

3816547290
None
